In [15]:
with open('/Users/nikahelicopter/Desktop/GERMANY/propaganda/propaganda_keywords.tsv','r') as keywords:
    keywords=list(keywords)
    keywords= [t.lower().replace('\n',' ').split('\t') for t in keywords]
    headers=[e[2] for e in keywords]
    headers.append('text')
    headers.append('length')
    headers.append('label')
import glob
import pymorphy2
import simplemma
from simplemma import text_lemmatizer
from simplemma import simple_tokenizer
langdata = simplemma.load_data('uk')
morph = pymorphy2.MorphAnalyzer(lang='uk')
import pandas as pd
import re
import spacy
nlp = spacy.load("ru_core_news_lg")

#'ukrainan_pro_russian'
subfolders=[1,2,3,4,5,6,7,8]
#subfolders=[1]
folders=['english','news_russian','french','romanian','ukrainian_pro_ukr']
folders=['ukrainan_pro_russian']

for folder in folders:
    for name in glob.glob('/Users/nikahelicopter/Desktop/GERMANY/propaganda/'+folder+'/'):
        print(name,'language')
        for sub in subfolders:
            print(sub)
            for lan in glob.glob('/Users/nikahelicopter/Desktop/GERMANY/propaganda/'+folder+'/'+str(sub)+'/*'):
                if lan.split('/')[-1]=='ukr':
                    print(lan,'ukr')
                else:
                    print(lan,'rus')
                    check=[u.split('.')[-1] for u in glob.glob(lan+'/*.csv')]
                    if check!=[]:
                        print('yes rus')
                        texts=[]
                        sources=[]
                        summary=[]
                        titles=[]
                        for name_2 in glob.glob(lan+'/*.csv'):
                            print(name_2,'csv')
                            titanic = pd.read_csv(name_2)
                            texts.append(titanic['Text'].values.tolist())
                            sources.append(titanic['Source'].values.tolist())
                            summary.append(titanic['Summary'].values.tolist())
                            titles.append(titanic['Title'].values.tolist())
                        texts=[[t.replace('\n',' ') for t in f] for f in texts]
                        sources=[[t.replace('\n',' ') for t in f] for f in sources] 
                        sources=[[t.replace('\n',' ').split('/')[-2:] for t in f] for f in sources] 
                        summary=[[t.replace('\n',' ') for t in f] for f in summary]
                        titles=[[t.replace('\n',' ') for t in f] for f in titles]
                        final=[]
                        lab=[]
                        for e, l in zip(texts,sources):
                            for t in range(len(e)):
                                final.append(e[t])
                                lab.append(l[t])
                        english=[]
                        labels=[]
                        for name_2 in glob.glob(lan+'/*.tsv'):
                            print(name_2)
                            with open(name_2,'r') as inp:
                                inp=list(inp)
                                inp=[e.replace('\n','').split('\t') for e in inp[1:]]
                                english.append(inp)
                                labels.append(name_2.split('/')[-1])
                        labels2=[]
                        texts=[]
                        sources=[]
                        summary=[]
                        titles=[]
                        for e,l in zip(english, labels):
                            for text in range (len(e)):
                                texts.append(e[text][2])
                                sources.append(e[text][5])
                                summary.append(e[text][-3])
                                titles.append(e[text][0])
                                labels2.append(l)

                        print(len(final),len(texts))
                        final.extend(texts)
                        lab.extend(labels2)
                        print(len(final))
                        tokens=[]
                        lemma=[]
                        posu=[]
                        dependancy=[]
                        final_labels=[]
                        full_text=[]
                        stop_words=[]
                        numbers=[]
                        for e in range(len(final)):
                            doc=nlp(final[e])
                            t=[[token.text for token in sent] for sent in doc.sents]
                            if t not in tokens:
                                if lab[e][0]=='':
                                    labo=lab[e][1]
                                else:
                                    labo=lab[e][0]
                                final_labels.append(labo)
                                full_text.append(final[e])
                                tokens.append(t)
                                posu.append([token.pos_ for token in doc] )
                                lemma.append([token.lemma_.lower() for token in doc ])
                                dependancy.append([token.dep_ for token in doc ])
                                stop_words.append([token.is_stop for token in doc ])
                                numbers.append([token.is_alpha for token in doc])
                        length=[]
                        for e in lemma:
                            length.append(len(e))
                        from statistics import mean
                        roman_stative=['placă', 'știu', 'aparține', 'iubi', 'realiza', 'se potrivesc', 'urăsc', 'presupun', 'conține', 'vrea', 'însemne', 'constau', 'nevoie', 'înțelege', 'părea', 'prefera', 'crede', 'depinde', 'acord', 'amintiți-vă', 'contează', 'minte', 'recunoaște', 'vezi', 'dețină', 'apare', 'privi', 'sunet', 'gust', 'miros', 'auzi', 'uimiți', 'nega', 'acord', 'mulțumi', 'impresiona', 'satisface', 'promite', 'surprinde', 'îndoială', 'gândi', 'simți', 'doresc', 'imagina', 'îngrijorare', 'aversiune', 'fi', 'avea', 'merita', 'implica', 'include', 'lipsește', 'măsura', 'poseda', 'dator', 'cântări'
                                      ]
                        english_stative=['like', 'know', 'belong', 'love', 'realise', 'fit', 'hate', 'suppose', 'contain', 'want', 'mean', 'consist', 'need', 'understand', 'seem', 
                        'prefer', 'believe', 'depend', 'agree', 'remember', 'matter', 'mind', 'recognise', 'see', 'own', 'appear', 'look', 'sound', 'taste', 'smell', 'hear', 'astonish', 'deny', 'disagree', 'please', 'impress', 'satisfy', 'promise', 'surprise', 'doubt', 'think',
                        'feel', 'wish', 'imagine', 'concern', 'dislike', 'be', 'have', 'deserve', 'involve', 'include', 'lack', 'measur', 'possess', 'owe', 'weigh']
                        russian_stative=['любить', 'знать', 'принадлежать', 'нравиться', 'понимать', 'соответствовать', 'ненавидеть', 'предполагать', 'содержать', 'хотеть', 'означать', 'состоять', 'понимать', 'казаться', 'предпочитать', 'верить', 'зависеть', 'соглашаться', 'помнить', 'значение', 'разум', 'узнавать', 'видеть', 'собственный', 'казаться', 'выглядеть', 'звучать', 'вкус', 'запах', 'слышать', 'поражать', 'отрицать', 'угодить', 'производить', 'удовлетворять', 'обещать', 'удивить', 'сомневаться', 'думать', 'чувствовать', 'желать', 'воображать', 'беспокоиться', 'быть', 'иметь', 'заслуживать', 'вовлекать', 'включать', 'отсутствовать', 'обладать', 'весить']
                        ukr_stative=['кохати', 'знати', 'належати', 'подобатися', 'розуміти', 'відповідати', 'ненавидіти', 'припускати', 'утримувати', 'хотіти', 'означати', 'перебувати', 'розуміти', 'здаватися', 'перевагу', 'вірити', 'залежати', 'погоджуватися', "пам'ятати", 'значення', 'дізнаватися', 'бачити', 'власний', 'здаватися', 'виглядати', 'звучати', 'смак', 'запах', 'чути', 'вражати', 'заперечувати', 'догодити', 'задовольняти', 'обіцяти', 'здивувати', 'сумніватися', 'думати', 'відчувати', 'бажати', 'уявляти', 'турбуватися', 'бути', 'мати', 'заслуговувати', 'залучати', 'включати', 'відсутня', 'мати', 'важити']
                        quo=['“','”', """" """,'‘','’',"'",'«','»']
                        bbt=['but','но','але', 'проте']
                        neg=['no','nothing','nobody','nowhere','не','нема','ни','ні','нигде','ничего','нічого', 'ніде', 'ніхто','нікому','нікого' ,'ніщо', 'никто', 'немає','nu','nimic','nicăieri','nimeni']
                        survey_words=['analysis','audit','check','inquiry','inspection','review','study','overview','survey','poll','interview',
                                     'анализ','аудит','проверка','расследование','инспекция','обзор','исследование','обзор','опрос','интервью',
                                     "analiză", "audit", "verificare", "anchetă", "inspecție", "revizuire", "studiu", "anchetă", "sondaj", "interviu",
                                     "аналіз", "аудит", "перевірк", "запит", "інспекці", "огляд", "досліджен", "опитування", "інтерв'ю"]
                        reported=['add', 'announce', 'answer', 'claim', 'convey', 'declare', 'condemn', 'deliver', 'disclose', 'estimate', 'express', 'mention', 'repeat', 'reply', 'respond', 'reveal', 'speak', 'say', 'state', 'suggest', 'tell', 'affirm', 'allege', 'communicate', 'imply', 'judge', 'pronounce', 'report',
                        'adăugați', 'anunță', 'răspunde', 'revendică', 'transmite', 'declara', 'condamna', 'livra', 'dezvăluie', 'estima', 'exprima', 'menționa', 'repetați', 'răspunde', 'răspunde', 'dezvăluie', 'vorbi', 'spune', 'declară', 'sugerează', 'spune', 'afirma', 'susține', 'comunica', 'implica', 'judecător', 'pronunță', 'raport',
                        'добавить', 'объявить', 'ответить', 'претензия', 'передавать', 'объявить', 'осудить', 'доставить', 'раскрывать', 'оценить', 'выразить', 'упоминать', 'повторить', 'ответить', 'ответить', 'раскрывать', 'говорить', 'сказать', 'заявить', 'предлагать', 'рассказать', 'утверждать', 'утверждать', 'сообщать',
                        'подразумевать', 'судить', 'произносить', 'сообщать','добавить', 'объявить', 'ответить', 'претензия', 'передавать', 'объявить', 'осудить', 'доставить', 'раскрывать', 'оценить', 'выразить', 'упоминать', 'повторить', 'ответить', 'раскрывать', 'говорить', 'сказать', 'заявить', 'предлагать', 'рассказать', 'утверждать',
                        'утверждать', 'сообщать', 'подразумевать', 'судить', 'произносить'
                        'дода', 'оголоси', 'відпові', 'претензі', 'переда', 'оголоси', 'засуджу', 'оцін', 'згада', 'повтори',   'розкри', 'говори', 'сказа', 'запропонув', 'розпові', 'стверджувa', 'спілкува', 'увазі', 'вимовля', 'звіт']
                        compare=['more','більш','более', 'менее', 'меньш','same', 'mai', 'puțin','fel' ]
                        #ask 
                        superlative=['most','найбільш','наиболее','наименее', 'найменш','cei', 'cele', 'cea','cel']
                        #-іш-  / ее /ей 
                        modals=['should','shall','may','might','can','could','must','cannot','повин','може','зможе','зміг', 'змогла', 'зможуть', 'можуть', 'могу', 'змогу', 'зможеш', 'можеш', 'можете', 'зможете', 'мочь', 'должен','должна', 'должны', 'сможет', 'смогут', 'смогу', 'сможем', 'сможете', 'смог', 'смогла', 'смогли',
                        'trebuie','trebui','poate','putea']
                        # вы в румынском
                        you=['you','ти','ты','tu','ви','вы']
                        me=['i','я','eu']
                        wetheyheshe=['i','я','eu','you','ти','ты','tu','ви','вы','we','they','she','he','it','ми' ,'вони' ,'вона' ,'воно','він','он' ,'она' ,'оно' ,'они' ,'мы','noi' ,'ei' ,'el' ,'ea' ]
                        money=['$','£','€','₴','₽','euros','dollars','rubles','hrivnas','leu','rublă','grivne','euro']
                        quotes=[sum([sum([1 for i in e if i in quo]) for e in t]) for t in tokens]

                        print(len(quotes))
                        stats=[sum([sum([1 for i in e if i=='%' or 'procent' in i or 'sută' in i]) for e in t]) for t in tokens]
                        print(len(stats))
                        but=[sum([sum([1 for i in e if i.lower() in bbt]) for e in t]) for t in tokens]
                        print(len(but))
                        adj=[sum([e.count('ADJ') for e in t]) for t in posu]
                        print(len(adj))
                        advs=[sum([e.count('ADV') for e in t]) for t in posu]
                        print(advs)
                        negations=[sum([sum([1 for i in e if i.lower() in neg]) for e in t]) for t in tokens]
                        print(negations)
                        survey=[sum([t.count(w) for w in survey_words if w in t]) for t in lemma]
                        averagesentlen=[mean([len(s) for s in t]) for t in tokens]
                        proper=[sum([e.count('PROPN') for e in t]) for t in posu]
                        report=[sum([t.count(w) for w in reported if w in t]) for t in lemma]
                        conj=[sum([e.count('CCONJ')+e.count('SCONJ') for e in t]) for t in posu]
                        comparative=[sum([t.count(w) for w in compare if w in t]) for t in lemma]
                        superlat=[sum([t.count(w) for w in superlative if w in t]) for t in lemma]
                        print(report,'rep')
                        print(proper)
                        print(averagesentlen)
                        print(survey)
                        print(conj)
                        print(comparative)
                        print(superlat)

                        questions=[sum([sum([1 for i in e if i=='?']) for e in t]) for t in tokens]
                        print(questions)
                        u=[sum([sum([1 for i in e if i.lower() in you]) for e in t]) for t in tokens]
                        print(u)
                        meme=[sum([sum([1 for i in e if i.lower() in me]) for e in t]) for t in tokens]
                        print(meme)
                        personal=[sum([sum([1 for i in e if i.lower() in wetheyheshe]) for e in t]) for t in tokens]
                        print(personal)
                        mon=[sum([sum([1 for i in e if i.lower() in money]) for e in t]) for t in tokens]
                        print(mon)
                        modal=[sum([t.count(w) for w in modals if w in t]) for t in lemma]
                        statives=[sum([t.count(w) for w in english_stative+russian_stative+ukr_stative+roman_stative if w in t]) for t in lemma]
                        print(modal)
                        print(statives)
                        fear=[]
                        surprise=[]
                        anger=[]
                        anticipation=[]
                        disgust=[]
                        joy=[]
                        negative=[]
                        positive=[]
                        sadness=[]
                        trust=[]
                        oo=[fear,surprise,anger,anticipation,disgust,joy,positive,negative,sadness,trust]
                        j=['fear','surprise','anger','anticipation','disgust','joy','positive','negative','sadness','trust']
                        for e in range(len(j)):
                            print(j[e])
                            with open('/Users/nikahelicopter/Desktop/GERMANY/propaganda/rus_lexicon/'+j[e]+'rus.txt','r') as inp:
                                inp=list(inp)
                                inp=inp[0].replace('\n','').split(', ')
                                oo[e].append(inp)
                        strah=[sum([t.count(w) for w in list(set(fear[0])) if w in t]) for t in lemma]
                        udivlen=[sum([t.count(w) for w in list(set(surprise[0])) if w in t]) for t in lemma]
                        zol=[sum([t.count(w) for w in list(set(anger[0])) if w in t]) for t in lemma]
                        jdu=[sum([t.count(w) for w in list(set(anticipation[0])) if w in t]) for t in lemma]
                        fu=[sum([t.count(w) for w in list(set(disgust[0])) if w in t]) for t in lemma]
                        ura=[sum([t.count(w) for w in list(set(joy[0])) if w in t]) for t in lemma]
                        negati=[sum([t.count(w) for w in list(set(negative[0])) if w in t]) for t in lemma]
                        positi=[sum([t.count(w) for w in list(set(positive[0])) if w in t]) for t in lemma]
                        grustno=[sum([t.count(w) for w in list(set(sadness[0])) if w in t]) for t in lemma]
                        doverie=[sum([t.count(w) for w in list(set(trust[0])) if w in t]) for t in lemma]    
                        with open('/Users/nikahelicopter/Desktop/GERMANY/propaganda/abstract.tsv','r') as abstract:
                            abstr=list(abstract)
                            abstr=[e.replace('\n','').split('\t') for e in abstr]
                        with open('/Users/nikahelicopter/Desktop/GERMANY/propaganda/assertion.tsv','r') as assertion:
                            assertion=list(assertion)
                            assertion=[e.replace('\n','').split('\t') for e in assertion]  
                        eng_abstr=[e[0].lower() for e in abstr]
                        rus_abstr=[e[1].lower() for e in abstr]
                        rom_abstr=[e[2].lower() for e in abstr]
                        ukr_abstr=[e[3].lower() for e in abstr]
                        eng_assert=[e[0].lower() for e in assertion]
                        rus_assert=[e[3].lower() for e in assertion]
                        rom_assert=[e[2].lower() for e in assertion]
                        ukr_assert=[e[1].lower() for e in assertion]
                        abtract=eng_abstr+rus_abstr+rom_abstr+ukr_abstr
                        asser=eng_assert+rus_assert+rom_assert+ukr_assert
                        abstract_nouns=[sum([t.count(w) for w in abtract if w.lower() in t]) for t in lemma]
                        assertion_words=[sum([t.count(w) for w in asser if w.lower() in t]) for t in lemma]
                        dd=['фактически', 'возможно', 'более того', 'Однако', 'несомненно', 'одновременно', 'в таком случае', 'очень сильно', 'первый', 'вместо', 'в основном', 'в общем', 'в действительности', 'например', 'пример', 'по сравнению с', 'по сравнению с', 'с одной стороны', 'с одной стороны', 'с другой стороны', 'с другой стороны', 'в отличие от первого', 'во-вторых', 'в-третьих', 'в данном контексте', 'В заключение', 'в конце концов', 'лаконично', 'к точке', 'начать с', 'Однако', 'тем не менее', 'потом', 'после', 'вследствие этого', 'в конце концов', 'превыше всего']
                        add_modal2=['фактически', 'категорически', 'определенно', 'несомненно', 'точно', 'положительно', 'точно', 'В самом деле', 'безусловно', 'действительно', 'безусловно', 'бесспорно', 'окончательно', 'решительно', 'решительно', 'просто', 'однозначно', 'последовательно', 'постоянно', 'часто', 'иногда', 'с интервалами', 'время от времени', 'временами', 'здесь и там', 'спорадически', 'сейчас и тогда', 'выключить и включить', 'по случаю', 'сейчас и тогда', 'периодически', 'периодически', 'едва', 'едва', 'нерегулярно', 'редко', 'Больше никогда', 'особенно', 'необычайно', 'очень сильно', 'тонко', 'Больше никогда', 'маленький', 'замечательный', 'заметный', 'Почти никогда', 'уникальный', 'необычный', 'необычный', 'нерегулярный', 'не часто', 'необычный', 'жирный', 'категоричный', 'ясно и отчетливо', 'определенный', 'недвусмысленный', 'недвусмысленный', 'очевидный', 'заметный', 'положительный', 'точный', 'отчетливый', 'специфический', 'простой', 'недвусмысленный', 'недвусмысленный', 'четко определен', 'слышимый', 'ясно и отчетливо', 'полный', 'хрустящий', 'определенный', 'различимый', 'простой', 'явный', 'исправлено', 'тупой', 'полный', 'графика', 'острый', 'отмечен', 'точный', 'не расплывчатый', 'особый', 'отчетливый', 'звон', 'сильный', 'острый', 'силуэт', 'заметный', 'нечеткий', 'видимый', 'яркий', 'хорошо обоснованный', 'хорошо помечены', 'уверенная в себе', 'положительный', 'заверенный', 'напористый', 'Конечно', 'веря', 'спокойствие', 'Конечно', 'беспрекословный', 'сангвиник', 'содержание', 'безопасный', 'уверенная в себе', 'равнодушный', 'невозмутимый', 'беззаботный', 'равнодушный', 'невозмутимый', 'непоколебимый', 'очевидный', 'определенный', 'определенный', 'отчетливый', 'очевидный', 'очевидный', 'Чисто', 'положительный', 'точный', 'очевидный', 'определенный', 'несомненно', 'остро', 'слышно', 'несомненно', 'безошибочно', 'решительно', 'различимо', 'бесспорно', 'несомненно', 'правдоподобно', 'явно', 'поразительно', 'заметно', 'манифест', 'очевидный', 'всепроникающий', 'ощутимый', 'поразительный', 'чистый', 'узнаваемый', 'острый', 'резонансный', 'фундаментальный', 'решающий', 'ключевой', 'базовый', 'незаменимый', 'незаменимый', 'императив', 'необходимо', 'первостепенный', 'обязательный', 'важный', 'неизбежный', 'срочный', 'жизненно важный', 'связывание', 'кардинал', 'главный', 'целесообразный', 'предварительное условие', 'нажатие', 'начальный', 'главный', 'необходимо', 'довольно важный', 'Нижняя линия', 'императив', 'обязательный', 'требуемый этикетом', 'элементарный', 'императив', 'действующий', 'значительный', 'название игры', 'необходимо', 'обязательный', 'квинтэссенция', 'заявил', 'желанный', 'предотвратимый', 'бесполезный', 'безвозмездный', 'не имеющий отношения', 'ненужный', 'лишний', 'ненужный', 'бесполезный', 'бесполезный', 'случайный', 'дополнительный', 'случайный', 'случайный', 'безвозмездный', 'случайный', 'необязательный', 'лишний', 'непомерный', 'необязательный', 'необязательный', 'внешний', 'несущественный', 'расточительный', 'необязательный', 'несущественный', 'необязательный', 'расточительный', 'щедрый', 'произвольный', 'лишний', 'лишний', 'непрошенный', 'некритичный', 'нежелательный', 'несущественный', 'нежелательный']
                        clamy_words2=['истинный', 'определенный', 'точка зрения', 'противоречие', 'вера', 'сомневаться', 'сохранять', 'доверять', 'считать', 'масса', 'полагать', 'истинный', 'полагаться', 'определенный', 'точка зрения', 'противоречить', 'доверие', 'Вера', 'сомневаться', 'учетная запись', 'считать', 'масса', 'Конечно', 'убежденный', 'доверчивый', 'мнение', 'считать', 'факт', 'определенный', 'убежденный', 'доверчивый', 'мнение', 'факт', 'сомневаться', 'предполагать', 'созерцать', 'ожидать', 'Чувствовать', 'подозревать', 'судить', 'распознавать', 'видеть', 'заключить', 'рассмотреть возможность', 'представлять себе', 'ценить', 'ценить', 'представлять себе', 'подозревать', 'созерцать', 'Чувствовать', 'не согласен', 'предполагать', 'подозревать', 'убежденный', 'полагать', 'принимать', 'признавать', 'созерцать', 'подозревать', 'доверять', 'подтверждать', 'постулат', 'предполагают', 'полагать', 'клясться', 'ожидать', 'предположение', 'спекулировать', 'предполагать', 'сознавать', 'признавать', 'разрешать', 'уступать', 'сознавать', 'признавать', 'сознавать', 'соглашаться', 'спорить', 'дебаты', 'бросать вызов', 'спор', 'противоречить', 'спор', 'противостоять', 'борьба', 'акция протеста', 'оказывать сопротивление', 'противоречить', 'прилавок', 'не одобрять', 'не согласен', 'противоречить', 'противоречить', 'не согласен', 'расходиться', 'не согласен', 'прилавок', 'несогласие', 'мнение', 'правильный', 'правдивый', 'фактический', 'точный', 'законный', 'заслуживающий доверия', 'неоспоримый', 'несомненный', 'неоспоримый', 'правдивый', 'правдивый', 'правдивый', 'оценка', 'предположение', 'отношение', 'вывод', 'чувство', 'идея', 'впечатление', 'суждение', 'разум', 'воображение', 'Посмотреть', 'реакция', 'чувство', 'предположение', 'теория', 'мысль', 'Посмотреть', 'Посмотреть', 'воображение', 'оценивать', 'оценивать', 'гипотеза', 'склонность', 'вывод', 'убеждение', 'постулат', 'предположение', 'презумпция', 'предположение', 'предположение', 'подозрение', 'утверждение', 'мой', 'заверенный', 'Чисто', 'определенный', 'убедительный', 'Вне всяких сомнений', 'подлинный', 'неоспоримый', 'неоспоримый', 'убежденный', 'непоколебимый', 'стойкий', 'неизменный', 'неизменный', 'несомненный', 'недвусмысленный', 'непогрешимый', 'несгибаемый', 'неквалифицированный', 'неоспоримый', 'несомненный', 'непоколебимый', 'действительный']
                        dd=list(set(dd))
                        add_modal2=list(set(add_modal2))
                        clamy_words2=list(set(clamy_words2))
                        modality_words=[sum([' '.join(t).count(w) for w in add_modal2 if w in t]) for t in lemma]
                        discourse_markers=[]
                        temp=[]
                        claims=[]
                        temc=[]
                        for e in tokens:
                            for sent in e:
                                s=' '.join([e.lower() for e in sent])
                                find=sum([s.count(w) for w in dd if w in s])
                                temp.append(find)
                                cl=sum([s.count(w) for w in clamy_words2 if w in s])
                                temc.append(cl)
                            discourse_markers.append(sum(temp))
                            claims.append(sum(temc))
                            temc=[]
                            temp=[]
                        consess2=['хотя', 'несмотря на то что']
                        reason2=['потому что', 'поскольку', 'так как', 'следовательно', 'таким образом']
                        purpose2=['так', 'так что', 'таким образом', 'чтобы', 'для того, чтобы']
                        relative2=['который', 'тот', 'чей', 'кого', 'чей']
                        time2=['до', 'после', 'когда', 'пока', 'до', 'поскольку', 'как только', 'однажды', 'всякий раз, когда', 'теперь, когда']
                        condition2=['в случае', 'только если', 'полагая', 'будь то', 'если']


                        cons=[]
                        reas=[]
                        purp=[]
                        relat=[]
                        tim=[]
                        cond=[]
                        temc=[]
                        tempr=[]
                        tempp=[]
                        temprel=[]
                        tempti=[]
                        tempcond=[]
                        for e in tokens:
                            for sent in e:
                                s=' '.join([e.lower() for e in sent])
                                co=sum([s.count(w) for w in consess2 if w in s])
                                re=sum([s.count(w) for w in reason2 if w in s])
                                pu=sum([s.count(w) for w in purpose2 if w in s])
                                rel=sum([s.count(w) for w in relative2 if w in s])
                                ti=sum([s.count(w) for w in time2 if w in s])
                                con=sum([s.count(w) for w in condition2 if w in s])
                                temc.append(co)
                                tempr.append(re)
                                tempp.append(pu)
                                temprel.append(rel)
                                tempti.append(ti)
                                tempcond.append(con)
                            cons.append(sum(temc))
                            reas.append(sum(tempr))
                            purp.append(sum(tempp))
                            relat.append(sum(temprel))
                            tim.append(sum(tempti))
                            cond.append(sum(tempcond))
                            temc=[]
                            tempr=[]
                            tempp=[]
                            temprel=[]
                            tempti=[]
                            tempcond=[]

                        passive=[]
                        temp=0
                        for p,l,t in zip(posu,lemma,tokens):
                            for word in range(len(l)):
                                if l[word]=='быть':
                                    print(l[word])
                                    if p[word+1]=='VERB':
                                        print(l[word]+l[word+1])
                                        temp+=1
                                    elif len(l)>=word+3:
                                        if p[word+1]=='ADV' and p[word+2]=='VERB':
                                            temp+=1
                            passive.append(temp)
                            temp=0
                        key=[]
                        for e in lemma:
                            find=[sum([' '.join(e).count(l) for l in w]) for w in keywords]
                            key.append([str(g) for g in find])
                        with open('russianukr_'+str(sub)+'.tsv','w') as out:
                                out.write('\t'.join(['labels','quotes', 'stats', 'but', 'adj', 'advs', 'negations', 'survey', 'averagesent', 'proper', 'report', 'conj', 'comparative', 'superlat', 'questions', 'meme', 'personal', 'mon', 'modal', 'statives', 'strah', 'udivlen', 'zol', 'jdu', 'fu', 'ura', 'negati', 'positi', 'grustno', 'doverie', 'abstract_nouns', 'assertion_words', 'discourse_markers', 'claims', 'modality_words', 'cons', 'reas', 'purp', 'relat', 'tim', 'cond', 'passive','lentok'])+'\t'+'\t'.join(headers)+'\n')
                                for e in range(len(passive)):
                                    out.write('\t'.join([final_labels[e],str(quotes[e]),str(stats[e]),str(but[e]),str(adj[e]),str(advs[e]),str(negations[e]),str(survey[e]),str(averagesentlen[e]),str(proper[e]),str(report[e]),str(conj[e]),str(comparative[e]),str(superlat[e]),str(questions[e]),str(meme[e]),str(personal[e]),str(mon[e]),str(modal[e]),str(statives[e]),str(strah[e]),str(udivlen[e]),str(zol[e]),str(jdu[e]),str(fu[e]),str(ura[e]),str(negati[e]),str(positi[e]),str(grustno[e]),str(doverie[e]),str(abstract_nouns[e]),str(assertion_words[e]),str(discourse_markers[e]),str(claims[e]),str(modality_words[e]),str(cons[e]),str(reas[e]),str(purp[e]),str(relat[e]),str(tim[e]),
                                                         str(cond[e]),str(passive[e]),str(length[e])])+'\t'+'\t'.join(key[e])+'\t'+full_text[e]+'\n')


                                
                            
                            
                    else:
                        english=[]
                        labels=[]
                        for name_2 in glob.glob(lan+'/*.tsv'):
                            print(name_2,'tsv only')
                            with open(name_2,'r') as inp:
                                inp=list(inp)
                                inp=[e.replace('\n','').split('\t') for e in inp[1:]]
                                english.append(inp)
                                labels.append(name_2.split('/')[-1])

                        print(len(labels))
                        print(len(english))
                        labels2=[]
                        texts=[]
                        sources=[]
                        summary=[]
                        titles=[]
                        for e,l in zip(english, labels):
                            for text in range (len(e)):
                                texts.append(e[text][2])
                                sources.append(e[text][5])
                                summary.append(e[text][-3])
                                titles.append(e[text][0])
                                labels2.append(l)

                        tokens=[]
                        posu=[]
                        lemma=[]
                        dependancy=[]
                        stop_words=[]
                        numbers=[]
                        final_labels=[]
                        full_text=[]
                        for e in range(len(texts)):
                            doc=nlp(texts[e])
                            t=[[token.text for token in sent] for sent in doc.sents]
                            if t not in tokens:
                                if labels2[e][0]=='':
                                    labo=labels2[e][1]
                                else:
                                    labo=labels2[e][0]
                                final_labels.append(labels2[e])
                                full_text.append(texts[e])
                                tokens.append(t)
                                posu.append([token.pos_ for token in doc] )
                                lemma.append([token.lemma_.lower() for token in doc ])
                                dependancy.append([token.dep_ for token in doc ])
                                stop_words.append([token.is_stop for token in doc ])
                                numbers.append([token.is_alpha for token in doc])

                        length=[]
                        for e in lemma:
                            length.append(len(e))

                        from statistics import mean
                        roman_stative=['placă', 'știu', 'aparține', 'iubi', 'realiza', 'se potrivesc', 'urăsc', 'presupun', 'conține', 'vrea', 'însemne', 'constau', 'nevoie', 'înțelege', 'părea', 'prefera', 'crede', 'depinde', 'acord', 'amintiți-vă', 'contează', 'minte', 'recunoaște', 'vezi', 'dețină', 'apare', 'privi', 'sunet', 'gust', 'miros', 'auzi', 'uimiți', 'nega', 'acord', 'mulțumi', 'impresiona', 'satisface', 'promite', 'surprinde', 'îndoială', 'gândi', 'simți', 'doresc', 'imagina', 'îngrijorare', 'aversiune', 'fi', 'avea', 'merita', 'implica', 'include', 'lipsește', 'măsura', 'poseda', 'dator', 'cântări'
                                      ]
                        english_stative=['like', 'know', 'belong', 'love', 'realise', 'fit', 'hate', 'suppose', 'contain', 'want', 'mean', 'consist', 'need', 'understand', 'seem', 
                        'prefer', 'believe', 'depend', 'agree', 'remember', 'matter', 'mind', 'recognise', 'see', 'own', 'appear', 'look', 'sound', 'taste', 'smell', 'hear', 'astonish', 'deny', 'disagree', 'please', 'impress', 'satisfy', 'promise', 'surprise', 'doubt', 'think',
                        'feel', 'wish', 'imagine', 'concern', 'dislike', 'be', 'have', 'deserve', 'involve', 'include', 'lack', 'measur', 'possess', 'owe', 'weigh']
                        russian_stative=['любить', 'знать', 'принадлежать', 'нравиться', 'понимать', 'соответствовать', 'ненавидеть', 'предполагать', 'содержать', 'хотеть', 'означать', 'состоять', 'понимать', 'казаться', 'предпочитать', 'верить', 'зависеть', 'соглашаться', 'помнить', 'значение', 'разум', 'узнавать', 'видеть', 'собственный', 'казаться', 'выглядеть', 'звучать', 'вкус', 'запах', 'слышать', 'поражать', 'отрицать', 'угодить', 'производить', 'удовлетворять', 'обещать', 'удивить', 'сомневаться', 'думать', 'чувствовать', 'желать', 'воображать', 'беспокоиться', 'быть', 'иметь', 'заслуживать', 'вовлекать', 'включать', 'отсутствовать', 'обладать', 'весить']
                        ukr_stative=['кохати', 'знати', 'належати', 'подобатися', 'розуміти', 'відповідати', 'ненавидіти', 'припускати', 'утримувати', 'хотіти', 'означати', 'перебувати', 'розуміти', 'здаватися', 'перевагу', 'вірити', 'залежати', 'погоджуватися', "пам'ятати", 'значення', 'дізнаватися', 'бачити', 'власний', 'здаватися', 'виглядати', 'звучати', 'смак', 'запах', 'чути', 'вражати', 'заперечувати', 'догодити', 'задовольняти', 'обіцяти', 'здивувати', 'сумніватися', 'думати', 'відчувати', 'бажати', 'уявляти', 'турбуватися', 'бути', 'мати', 'заслуговувати', 'залучати', 'включати', 'відсутня', 'мати', 'важити']
                        quo=['“','”', """" """,'‘','’',"'",'«','»']
                        bbt=['but','но','але', 'проте']
                        neg=['no','nothing','nobody','nowhere','не','нема','ни','ні','нигде','ничего','нічого', 'ніде', 'ніхто','нікому','нікого' ,'ніщо', 'никто', 'немає','nu','nimic','nicăieri','nimeni']
                        survey_words=['analysis','audit','check','inquiry','inspection','review','study','overview','survey','poll','interview',
                                     'анализ','аудит','проверка','расследование','инспекция','обзор','исследование','обзор','опрос','интервью',
                                     "analiză", "audit", "verificare", "anchetă", "inspecție", "revizuire", "studiu", "anchetă", "sondaj", "interviu",
                                     "аналіз", "аудит", "перевірк", "запит", "інспекці", "огляд", "досліджен", "опитування", "інтерв'ю"]
                        reported=['add', 'announce', 'answer', 'claim', 'convey', 'declare', 'condemn', 'deliver', 'disclose', 'estimate', 'express', 'mention', 'repeat', 'reply', 'respond', 'reveal', 'speak', 'say', 'state', 'suggest', 'tell', 'affirm', 'allege', 'communicate', 'imply', 'judge', 'pronounce', 'report',
                        'adăugați', 'anunță', 'răspunde', 'revendică', 'transmite', 'declara', 'condamna', 'livra', 'dezvăluie', 'estima', 'exprima', 'menționa', 'repetați', 'răspunde', 'răspunde', 'dezvăluie', 'vorbi', 'spune', 'declară', 'sugerează', 'spune', 'afirma', 'susține', 'comunica', 'implica', 'judecător', 'pronunță', 'raport',
                        'добавить', 'объявить', 'ответить', 'претензия', 'передавать', 'объявить', 'осудить', 'доставить', 'раскрывать', 'оценить', 'выразить', 'упоминать', 'повторить', 'ответить', 'ответить', 'раскрывать', 'говорить', 'сказать', 'заявить', 'предлагать', 'рассказать', 'утверждать', 'утверждать', 'сообщать',
                        'подразумевать', 'судить', 'произносить', 'сообщать','добавить', 'объявить', 'ответить', 'претензия', 'передавать', 'объявить', 'осудить', 'доставить', 'раскрывать', 'оценить', 'выразить', 'упоминать', 'повторить', 'ответить', 'раскрывать', 'говорить', 'сказать', 'заявить', 'предлагать', 'рассказать', 'утверждать',
                        'утверждать', 'сообщать', 'подразумевать', 'судить', 'произносить'
                        'дода', 'оголоси', 'відпові', 'претензі', 'переда', 'оголоси', 'засуджу', 'оцін', 'згада', 'повтори',   'розкри', 'говори', 'сказа', 'запропонув', 'розпові', 'стверджувa', 'спілкува', 'увазі', 'вимовля', 'звіт']
                        compare=['more','більш','более', 'менее', 'меньш','same', 'mai', 'puțin','fel' ]
                        #ask 
                        superlative=['most','найбільш','наиболее','наименее', 'найменш','cei', 'cele', 'cea','cel']
                        #-іш-  / ее /ей 
                        modals=['should','shall','may','might','can','could','must','cannot','повин','може','зможе','зміг', 'змогла', 'зможуть', 'можуть', 'могу', 'змогу', 'зможеш', 'можеш', 'можете', 'зможете', 'мочь', 'должен','должна', 'должны', 'сможет', 'смогут', 'смогу', 'сможем', 'сможете', 'смог', 'смогла', 'смогли',
                        'trebuie','trebui','poate','putea']
                        # вы в румынском
                        you=['you','ти','ты','tu','ви','вы']
                        me=['i','я','eu']
                        wetheyheshe=['i','я','eu','you','ти','ты','tu','ви','вы','we','they','she','he','it','ми' ,'вони' ,'вона' ,'воно','він','он' ,'она' ,'оно' ,'они' ,'мы','noi' ,'ei' ,'el' ,'ea' ]
                        money=['$','£','€','₴','₽','euros','dollars','rubles','hrivnas','leu','rublă','grivne','euro']
                        quotes=[sum([sum([1 for i in e if i in quo]) for e in t]) for t in tokens]

                        print(len(quotes))
                        stats=[sum([sum([1 for i in e if i=='%' or 'procent' in i or 'sută' in i]) for e in t]) for t in tokens]
                        print(len(stats))
                        but=[sum([sum([1 for i in e if i.lower() in bbt]) for e in t]) for t in tokens]
                        print(len(but))
                        adj=[sum([e.count('ADJ') for e in t]) for t in posu]
                        print(len(adj))
                        advs=[sum([e.count('ADV') for e in t]) for t in posu]
                        print(advs)
                        negations=[sum([sum([1 for i in e if i.lower() in neg]) for e in t]) for t in tokens]
                        print(negations)
                        survey=[sum([t.count(w) for w in survey_words if w in t]) for t in lemma]
                        averagesentlen=[mean([len(s) for s in t]) for t in tokens]
                        proper=[sum([e.count('PROPN') for e in t]) for t in posu]
                        report=[sum([t.count(w) for w in reported if w in t]) for t in lemma]
                        conj=[sum([e.count('CCONJ')+e.count('SCONJ') for e in t]) for t in posu]
                        comparative=[sum([t.count(w) for w in compare if w in t]) for t in lemma]
                        superlat=[sum([t.count(w) for w in superlative if w in t]) for t in lemma]
                        print(report,'rep')
                        print(proper)
                        print(averagesentlen)
                        print(survey)
                        print(conj)
                        print(comparative)
                        print(superlat)

                        questions=[sum([sum([1 for i in e if i=='?']) for e in t]) for t in tokens]
                        print(questions)
                        u=[sum([sum([1 for i in e if i.lower() in you]) for e in t]) for t in tokens]
                        print(u)
                        meme=[sum([sum([1 for i in e if i.lower() in me]) for e in t]) for t in tokens]
                        print(meme)
                        personal=[sum([sum([1 for i in e if i.lower() in wetheyheshe]) for e in t]) for t in tokens]
                        print(personal)
                        mon=[sum([sum([1 for i in e if i.lower() in money]) for e in t]) for t in tokens]
                        print(mon)
                        modal=[sum([t.count(w) for w in modals if w in t]) for t in lemma]
                        statives=[sum([t.count(w) for w in english_stative+russian_stative+ukr_stative+roman_stative if w in t]) for t in lemma]
                        print(modal)
                        print(statives)
                        fear=[]
                        surprise=[]
                        anger=[]
                        anticipation=[]
                        disgust=[]
                        joy=[]
                        negative=[]
                        positive=[]
                        sadness=[]
                        trust=[]
                        oo=[fear,surprise,anger,anticipation,disgust,joy,positive,negative,sadness,trust]
                        j=['fear','surprise','anger','anticipation','disgust','joy','positive','negative','sadness','trust']
                        for e in range(len(j)):
                            print(j[e])
                            with open('/Users/nikahelicopter/Desktop/GERMANY/propaganda/rus_lexicon/'+j[e]+'rus.txt','r') as inp:
                                inp=list(inp)
                                inp=inp[0].replace('\n','').split(', ')
                                oo[e].append(inp)
                        strah=[sum([t.count(w) for w in list(set(fear[0])) if w in t]) for t in lemma]
                        udivlen=[sum([t.count(w) for w in list(set(surprise[0])) if w in t]) for t in lemma]
                        zol=[sum([t.count(w) for w in list(set(anger[0])) if w in t]) for t in lemma]
                        jdu=[sum([t.count(w) for w in list(set(anticipation[0])) if w in t]) for t in lemma]
                        fu=[sum([t.count(w) for w in list(set(disgust[0])) if w in t]) for t in lemma]
                        ura=[sum([t.count(w) for w in list(set(joy[0])) if w in t]) for t in lemma]
                        negati=[sum([t.count(w) for w in list(set(negative[0])) if w in t]) for t in lemma]
                        positi=[sum([t.count(w) for w in list(set(positive[0])) if w in t]) for t in lemma]
                        grustno=[sum([t.count(w) for w in list(set(sadness[0])) if w in t]) for t in lemma]
                        doverie=[sum([t.count(w) for w in list(set(trust[0])) if w in t]) for t in lemma]    
                        with open('/Users/nikahelicopter/Desktop/GERMANY/propaganda/abstract.tsv','r') as abstract:
                            abstr=list(abstract)
                            abstr=[e.replace('\n','').split('\t') for e in abstr]
                        with open('/Users/nikahelicopter/Desktop/GERMANY/propaganda/assertion.tsv','r') as assertion:
                            assertion=list(assertion)
                            assertion=[e.replace('\n','').split('\t') for e in assertion]  
                        eng_abstr=[e[0].lower() for e in abstr]
                        rus_abstr=[e[1].lower() for e in abstr]
                        rom_abstr=[e[2].lower() for e in abstr]
                        ukr_abstr=[e[3].lower() for e in abstr]
                        eng_assert=[e[0].lower() for e in assertion]
                        rus_assert=[e[3].lower() for e in assertion]
                        rom_assert=[e[2].lower() for e in assertion]
                        ukr_assert=[e[1].lower() for e in assertion]
                        abtract=eng_abstr+rus_abstr+rom_abstr+ukr_abstr
                        asser=eng_assert+rus_assert+rom_assert+ukr_assert
                        abstract_nouns=[sum([t.count(w) for w in abtract if w.lower() in t]) for t in lemma]
                        assertion_words=[sum([t.count(w) for w in asser if w.lower() in t]) for t in lemma]
                        dd=['фактически', 'возможно', 'более того', 'Однако', 'несомненно', 'одновременно', 'в таком случае', 'очень сильно', 'первый', 'вместо', 'в основном', 'в общем', 'в действительности', 'например', 'пример', 'по сравнению с', 'по сравнению с', 'с одной стороны', 'с одной стороны', 'с другой стороны', 'с другой стороны', 'в отличие от первого', 'во-вторых', 'в-третьих', 'в данном контексте', 'В заключение', 'в конце концов', 'лаконично', 'к точке', 'начать с', 'Однако', 'тем не менее', 'потом', 'после', 'вследствие этого', 'в конце концов', 'превыше всего']
                        add_modal2=['фактически', 'категорически', 'определенно', 'несомненно', 'точно', 'положительно', 'точно', 'В самом деле', 'безусловно', 'действительно', 'безусловно', 'бесспорно', 'окончательно', 'решительно', 'решительно', 'просто', 'однозначно', 'последовательно', 'постоянно', 'часто', 'иногда', 'с интервалами', 'время от времени', 'временами', 'здесь и там', 'спорадически', 'сейчас и тогда', 'выключить и включить', 'по случаю', 'сейчас и тогда', 'периодически', 'периодически', 'едва', 'едва', 'нерегулярно', 'редко', 'Больше никогда', 'особенно', 'необычайно', 'очень сильно', 'тонко', 'Больше никогда', 'маленький', 'замечательный', 'заметный', 'Почти никогда', 'уникальный', 'необычный', 'необычный', 'нерегулярный', 'не часто', 'необычный', 'жирный', 'категоричный', 'ясно и отчетливо', 'определенный', 'недвусмысленный', 'недвусмысленный', 'очевидный', 'заметный', 'положительный', 'точный', 'отчетливый', 'специфический', 'простой', 'недвусмысленный', 'недвусмысленный', 'четко определен', 'слышимый', 'ясно и отчетливо', 'полный', 'хрустящий', 'определенный', 'различимый', 'простой', 'явный', 'исправлено', 'тупой', 'полный', 'графика', 'острый', 'отмечен', 'точный', 'не расплывчатый', 'особый', 'отчетливый', 'звон', 'сильный', 'острый', 'силуэт', 'заметный', 'нечеткий', 'видимый', 'яркий', 'хорошо обоснованный', 'хорошо помечены', 'уверенная в себе', 'положительный', 'заверенный', 'напористый', 'Конечно', 'веря', 'спокойствие', 'Конечно', 'беспрекословный', 'сангвиник', 'содержание', 'безопасный', 'уверенная в себе', 'равнодушный', 'невозмутимый', 'беззаботный', 'равнодушный', 'невозмутимый', 'непоколебимый', 'очевидный', 'определенный', 'определенный', 'отчетливый', 'очевидный', 'очевидный', 'Чисто', 'положительный', 'точный', 'очевидный', 'определенный', 'несомненно', 'остро', 'слышно', 'несомненно', 'безошибочно', 'решительно', 'различимо', 'бесспорно', 'несомненно', 'правдоподобно', 'явно', 'поразительно', 'заметно', 'манифест', 'очевидный', 'всепроникающий', 'ощутимый', 'поразительный', 'чистый', 'узнаваемый', 'острый', 'резонансный', 'фундаментальный', 'решающий', 'ключевой', 'базовый', 'незаменимый', 'незаменимый', 'императив', 'необходимо', 'первостепенный', 'обязательный', 'важный', 'неизбежный', 'срочный', 'жизненно важный', 'связывание', 'кардинал', 'главный', 'целесообразный', 'предварительное условие', 'нажатие', 'начальный', 'главный', 'необходимо', 'довольно важный', 'Нижняя линия', 'императив', 'обязательный', 'требуемый этикетом', 'элементарный', 'императив', 'действующий', 'значительный', 'название игры', 'необходимо', 'обязательный', 'квинтэссенция', 'заявил', 'желанный', 'предотвратимый', 'бесполезный', 'безвозмездный', 'не имеющий отношения', 'ненужный', 'лишний', 'ненужный', 'бесполезный', 'бесполезный', 'случайный', 'дополнительный', 'случайный', 'случайный', 'безвозмездный', 'случайный', 'необязательный', 'лишний', 'непомерный', 'необязательный', 'необязательный', 'внешний', 'несущественный', 'расточительный', 'необязательный', 'несущественный', 'необязательный', 'расточительный', 'щедрый', 'произвольный', 'лишний', 'лишний', 'непрошенный', 'некритичный', 'нежелательный', 'несущественный', 'нежелательный']
                        clamy_words2=['истинный', 'определенный', 'точка зрения', 'противоречие', 'вера', 'сомневаться', 'сохранять', 'доверять', 'считать', 'масса', 'полагать', 'истинный', 'полагаться', 'определенный', 'точка зрения', 'противоречить', 'доверие', 'Вера', 'сомневаться', 'учетная запись', 'считать', 'масса', 'Конечно', 'убежденный', 'доверчивый', 'мнение', 'считать', 'факт', 'определенный', 'убежденный', 'доверчивый', 'мнение', 'факт', 'сомневаться', 'предполагать', 'созерцать', 'ожидать', 'Чувствовать', 'подозревать', 'судить', 'распознавать', 'видеть', 'заключить', 'рассмотреть возможность', 'представлять себе', 'ценить', 'ценить', 'представлять себе', 'подозревать', 'созерцать', 'Чувствовать', 'не согласен', 'предполагать', 'подозревать', 'убежденный', 'полагать', 'принимать', 'признавать', 'созерцать', 'подозревать', 'доверять', 'подтверждать', 'постулат', 'предполагают', 'полагать', 'клясться', 'ожидать', 'предположение', 'спекулировать', 'предполагать', 'сознавать', 'признавать', 'разрешать', 'уступать', 'сознавать', 'признавать', 'сознавать', 'соглашаться', 'спорить', 'дебаты', 'бросать вызов', 'спор', 'противоречить', 'спор', 'противостоять', 'борьба', 'акция протеста', 'оказывать сопротивление', 'противоречить', 'прилавок', 'не одобрять', 'не согласен', 'противоречить', 'противоречить', 'не согласен', 'расходиться', 'не согласен', 'прилавок', 'несогласие', 'мнение', 'правильный', 'правдивый', 'фактический', 'точный', 'законный', 'заслуживающий доверия', 'неоспоримый', 'несомненный', 'неоспоримый', 'правдивый', 'правдивый', 'правдивый', 'оценка', 'предположение', 'отношение', 'вывод', 'чувство', 'идея', 'впечатление', 'суждение', 'разум', 'воображение', 'Посмотреть', 'реакция', 'чувство', 'предположение', 'теория', 'мысль', 'Посмотреть', 'Посмотреть', 'воображение', 'оценивать', 'оценивать', 'гипотеза', 'склонность', 'вывод', 'убеждение', 'постулат', 'предположение', 'презумпция', 'предположение', 'предположение', 'подозрение', 'утверждение', 'мой', 'заверенный', 'Чисто', 'определенный', 'убедительный', 'Вне всяких сомнений', 'подлинный', 'неоспоримый', 'неоспоримый', 'убежденный', 'непоколебимый', 'стойкий', 'неизменный', 'неизменный', 'несомненный', 'недвусмысленный', 'непогрешимый', 'несгибаемый', 'неквалифицированный', 'неоспоримый', 'несомненный', 'непоколебимый', 'действительный']
                        dd=list(set(dd))
                        add_modal2=list(set(add_modal2))
                        clamy_words2=list(set(clamy_words2))
                        modality_words=[sum([' '.join(t).count(w) for w in add_modal2 if w in t]) for t in lemma]
                        discourse_markers=[]
                        temp=[]
                        claims=[]
                        temc=[]
                        for e in tokens:
                            for sent in e:
                                s=' '.join([e.lower() for e in sent])
                                find=sum([s.count(w) for w in dd if w in s])
                                temp.append(find)
                                cl=sum([s.count(w) for w in clamy_words2 if w in s])
                                temc.append(cl)
                            discourse_markers.append(sum(temp))
                            claims.append(sum(temc))
                            temc=[]
                            temp=[]
                        consess2=['хотя', 'несмотря на то что']
                        reason2=['потому что', 'поскольку', 'так как', 'следовательно', 'таким образом']
                        purpose2=['так', 'так что', 'таким образом', 'чтобы', 'для того, чтобы']
                        relative2=['который', 'тот', 'чей', 'кого', 'чей']
                        time2=['до', 'после', 'когда', 'пока', 'до', 'поскольку', 'как только', 'однажды', 'всякий раз, когда', 'теперь, когда']
                        condition2=['в случае', 'только если', 'полагая', 'будь то', 'если']


                        cons=[]
                        reas=[]
                        purp=[]
                        relat=[]
                        tim=[]
                        cond=[]
                        temc=[]
                        tempr=[]
                        tempp=[]
                        temprel=[]
                        tempti=[]
                        tempcond=[]
                        for e in tokens:
                            for sent in e:
                                s=' '.join([e.lower() for e in sent])
                                co=sum([s.count(w) for w in consess2 if w in s])
                                re=sum([s.count(w) for w in reason2 if w in s])
                                pu=sum([s.count(w) for w in purpose2 if w in s])
                                rel=sum([s.count(w) for w in relative2 if w in s])
                                ti=sum([s.count(w) for w in time2 if w in s])
                                con=sum([s.count(w) for w in condition2 if w in s])
                                temc.append(co)
                                tempr.append(re)
                                tempp.append(pu)
                                temprel.append(rel)
                                tempti.append(ti)
                                tempcond.append(con)
                            cons.append(sum(temc))
                            reas.append(sum(tempr))
                            purp.append(sum(tempp))
                            relat.append(sum(temprel))
                            tim.append(sum(tempti))
                            cond.append(sum(tempcond))
                            temc=[]
                            tempr=[]
                            tempp=[]
                            temprel=[]
                            tempti=[]
                            tempcond=[]

                        passive=[]
                        temp=0
                        for p,l,t in zip(posu,lemma,tokens):
                            for word in range(len(l)):
                                if l[word]=='быть':
                                    print(l[word])
                                    if p[word+1]=='VERB':
                                        print(l[word]+l[word+1])
                                        temp+=1
                                    elif len(l)>=word+3:
                                        if p[word+1]=='ADV' and p[word+2]=='VERB':
                                            temp+=1
                            passive.append(temp)
                            temp=0

                        key=[]
                        for e in lemma:
                            find=[sum([' '.join(e).count(l) for l in w]) for w in keywords]
                            print(len(find))
                            key.append([str(g) for g in find])
                        with open('russianukr_'+str(sub)+'.tsv','w') as out:
                                out.write('\t'.join(['labels','quotes', 'stats', 'but', 'adj', 'advs', 'negations', 'survey', 'averagesent', 'proper', 'report', 'conj', 'comparative', 'superlat', 'questions', 'meme', 
                                                     'personal', 'mon', 'modal', 'statives', 'strah', 'udivlen', 'zol', 'jdu', 'fu', 'ura', 'negati', 'positi', 'grustno', 'doverie',
                                                     'abstract_nouns', 'assertion_words', 'discourse_markers', 'claims', 'modality_words', 'cons', 'reas',
                                                     'purp', 'relat', 'tim', 'cond', 'passive','lentok'])+'\t'+'\t'.join(headers)+'\n')
                                for e in range(len(passive)):
                                    out.write('\t'.join([final_labels[e],str(quotes[e]),str(stats[e]),str(but[e]),str(adj[e]),str(advs[e]),str(negations[e]),str(survey[e]),str(averagesentlen[e]),str(proper[e]),str(report[e]),str(conj[e]),str(comparative[e]),str(superlat[e]),str(questions[e]),str(meme[e]),str(personal[e]),str(mon[e]),str(modal[e]),str(statives[e]),str(strah[e]),str(udivlen[e]),str(zol[e]),str(jdu[e]),str(fu[e]),str(ura[e]),str(negati[e]),str(positi[e]),str(grustno[e]),str(doverie[e]),str(abstract_nouns[e]),str(assertion_words[e]),str(discourse_markers[e]),str(claims[e]),str(modality_words[e]),str(cons[e]),str(reas[e]),str(purp[e]),str(relat[e]),str(tim[e]),
                                                         str(cond[e]),str(passive[e]),str(length[e])])+'\t'+'\t'.join(key[e])+'\t'+full_text[e]+'\n')



                       
                    
            

/Users/nikahelicopter/Desktop/GERMANY/propaganda/ukrainan_pro_russian/ language
1
/Users/nikahelicopter/Desktop/GERMANY/propaganda/ukrainan_pro_russian/1/ukr ukr
/Users/nikahelicopter/Desktop/GERMANY/propaganda/ukrainan_pro_russian/1/rus rus
yes rus
/Users/nikahelicopter/Desktop/GERMANY/propaganda/ukrainan_pro_russian/1/rus/my_scraped_vestiua.csv csv
/Users/nikahelicopter/Desktop/GERMANY/propaganda/ukrainan_pro_russian/1/rus/my_scraped_newseua.tsv
/Users/nikahelicopter/Desktop/GERMANY/propaganda/ukrainan_pro_russian/1/rus/my_scraped_strana2.tsv
/Users/nikahelicopter/Desktop/GERMANY/propaganda/ukrainan_pro_russian/1/rus/my_scraped_strana.tsv
/Users/nikahelicopter/Desktop/GERMANY/propaganda/ukrainan_pro_russian/1/rus/my_scraped_ukra.tsv
740 609
1349
523
523
523
523
[11, 15, 116, 125, 7, 14, 17, 131, 135, 65, 17, 52, 87, 53, 77, 4, 3, 7, 5, 3, 5, 11, 43, 12, 8, 18, 5, 8, 6, 3, 50, 54, 67, 51, 48, 73, 94, 31, 36, 33, 44, 45, 2, 21, 19, 7, 19, 18, 58, 7, 48, 278, 324, 292, 388, 359, 268, 21

[0, 0, 8, 4, 0, 0, 0, 3, 0, 1, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 4, 0, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0, 0, 1, 10, 16, 19, 30, 18, 13, 7, 1, 10, 28, 7, 21, 8, 17, 7, 10, 72, 18, 11, 11, 1, 11, 19, 17, 4, 20, 4, 4, 1, 10, 0, 5, 1, 2, 1, 9, 1, 4, 6, 5, 3, 4, 7, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 10, 4, 2, 9, 1, 2, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 4, 4, 5, 3, 3, 5, 6, 3, 3, 4, 3, 3, 5, 4, 6, 3, 0, 2, 2, 0, 0, 0, 1, 1, 2, 0, 0, 1, 0, 2, 3, 0, 4, 2, 3, 0, 2, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 20, 8, 2, 0, 6, 1, 2, 0, 3, 1, 4, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 10, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 19, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 3, 0, 0, 81, 14, 15, 22, 8, 18, 0, 0, 0

быть
бытьразговаривать
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
бытьустановить
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
бытьпоставить
быть
быть
быть
быть
быть
быть
быть
бытьиспользовать
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
бытьнаписать
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
бытьсоблюдаться
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
бытьотстроить
быть
быть
быть
быть
быть
быть
быть
быт

быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
бытьнадоесть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
бытьпривязать
быть
быть
быть
быть
быть
быть
бытьразрешить
быть
быть
быть
быть
быть
быть
быть
быть
бытьразвиваться
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
бытьсоздать
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
бытьпривлечь
быть
быть
бытьподписать
быть
бытьвыполнить
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
бытьпередать
быть
быть
быть
быть
быть
быть
быть
быть
бытьработать
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
бытьполучаться
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть
быть

[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 2, 1, 4, 2, 2, 2, 2, 3, 2, 4, 3, 2, 3, 3, 2, 2, 2, 3, 2, 6, 3, 3, 3, 5, 5, 2, 11, 5, 1, 1, 0, 2, 8, 2, 2, 3, 2, 0, 7, 14, 1, 1, 3, 2, 3, 4, 9, 3, 5, 2, 0, 2, 0, 7, 1, 2, 2, 18, 16, 4, 42, 13, 8, 1, 42, 1, 2, 2, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 9, 4, 0, 0, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 3, 0, 15, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 19, 2, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 2, 25, 0, 36, 28, 5, 2, 5, 10, 18, 22, 1, 17, 7, 1, 13, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0

...

116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116


[1, 3, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 2, 1, 0, 0, 6, 0, 1, 0, 0, 2, 0, 19, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 41, 2, 1, 0, 21, 0, 9, 38, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 3, 3, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 15, 4, 7, 0, 0, 4, 5, 10, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 7, 0, 8, 0, 0, 0, 0, 0, 3, 0, 0, 10, 0, 0, 3, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 1, 4, 0, 10, 16, 19, 30, 18, 13, 7, 1, 10, 28, 7, 21, 8, 17, 7, 10, 72, 18, 11, 11, 1, 11, 19, 17, 4, 20, 4, 4, 1, 1, 0, 2, 6, 18, 0, 3, 1, 0, 0, 0, 0, 1, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 5, 1, 0, 0, 0, 5, 1, 0, 0, 10, 0, 5, 1, 2, 1, 9, 1, 4, 6, 2, 3, 0, 1, 5, 3, 8, 8, 4, 7, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 3, 2, 1, 0, 1, 0, 0, 9]
[7, 12, 14, 2, 3, 3, 3, 2, 53, 4, 2, 3, 3, 2, 5, 10, 6, 2, 2, 2, 3, 3, 4, 5, 2, 2, 2, 1, 2, 2, 4, 4, 2, 29, 14, 7, 8, 22, 12, 11,

116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116


[2, 1, 0, 9, 2, 1, 0, 2, 0, 13, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0, 6, 0, 1, 0, 0, 2, 0, 19, 4, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 3, 3, 6, 0, 0, 0, 0, 0, 0, 0, 0, 1, 41, 2, 1, 0, 21, 0, 9, 38, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 1, 15, 4, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 7, 0, 4, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 8, 0, 0, 1, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 7, 1, 0, 10, 0, 10, 0, 0, 3, 0, 1, 0, 10, 0, 5, 1, 2, 1, 9, 1, 4, 6, 2, 3, 0, 1, 5, 3, 8, 8, 4, 7, 12, 0, 1, 4, 0, 1, 0, 0, 2, 3, 2, 1, 0, 10, 16, 19, 30, 18, 13, 7, 1, 10, 28, 7, 21, 8, 17, 7, 10, 72, 18, 11, 11, 1, 11, 19, 17, 4, 20, 4, 4, 1, 5, 1, 0, 0, 0, 5, 1, 0, 0, 0, 0]
[23, 7, 10, 61, 11, 17, 0, 22, 1, 53, 3, 2, 7, 4, 1, 6, 8, 2, 3, 1, 5, 2, 1, 1, 1, 4, 16, 4, 1, 2, 1, 2, 2, 4, 14, 7, 8, 22, 12, 1

116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116
116


In [6]:
print(len(posu),len(lemma),len(tokens))

523 523 523


In [9]:
print(word+2)

3982


In [8]:
print(len(l))

3982


In [11]:
len(l)>word+2

False